In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')


In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


# Cargo input

In [3]:
# Cargamos datos
pisos_20 = pd.read_csv("data origin/pisos_venta_2020.csv", delimiter=',') # cargamos fichero
pisos_21 = pd.read_csv("data origin/pisos_2021.csv", delimiter=',') # cargamos fichero

In [4]:
pisos_20['precio20']=pisos_20['precio']
pisos_21['precio21']=pisos_21['precio']

In [5]:
###### Esta funcion actualiza el número de instancias y features de mi modelo
def status_cols_rows (df):
    entries = df.shape[0]
    features =df.shape[1]
    print ('Nº de entradas:', entries)
    print ('Nº de features:', features)

In [6]:
status_cols_rows(pisos_20)
status_cols_rows(pisos_21)

Nº de entradas: 5520
Nº de features: 21
Nº de entradas: 4440
Nº de features: 26


### Observo los valores nulos

In [7]:
pisos_20.isnull().sum().sort_values(ascending=False)
pisos_21.isnull().sum().sort_values(ascending=False)

dist_centro                   658
long_                         658
lat_                          658
renta_media_barrio_persona      3
precio21                        0
n_habitaciones                  0
precio                          0
long                            0
lat                             0
metros                          0
planta                          0
tipo_vivienda                   0
n_banos                         0
ascensor_en_desc                0
desc                            0
ascensor_en_caract              0
trastero_en_desc                0
trastero_en_caract              0
garaje_en_desc                  0
garaje_en_caract                0
jardin_en_desc                  0
parquet_en_desc                 0
obra_nueva                      0
barrio                          0
ciudad                          0
titulo                          0
dtype: int64

#### Observo los tipos de cada variable

In [8]:
for c in pisos_20.columns:
    print('{}= {}'.format(c,pisos_20[c].dtype))

titulo= object
precio= object
metros= object
planta= object
tipo_vivienda= object
n_banos= object
n_habitaciones= object
ascensor_en_desc= int64
ascensor_en_caract= int64
trastero_en_desc= int64
trastero_en_caract= int64
garaje_en_desc= int64
garaje_en_caract= int64
jardin_en_desc= int64
parquet_en_desc= int64
obra_nueva= int64
barrio= object
ciudad= object
renta_media_barrio_persona= object
descripcion= object
precio20= object


## Transformaciones de algunas variables

#### Borro posibles instancias duplicadas

In [9]:
pisos_20=pisos_20.drop_duplicates()
pisos_21=pisos_21.drop_duplicates()

In [10]:
status_cols_rows(pisos_20)
status_cols_rows(pisos_21)

Nº de entradas: 5510
Nº de features: 21
Nº de entradas: 4413
Nº de features: 26


#### Creo un dataframe con los datos del 2020 que sigan en la inmobilidaria y del 2021 para comparar los precios. Lo usaré en R

In [11]:
pisos_20_21=pd.merge(pisos_20, pisos_21, how='inner', on=('metros','garaje_en_desc','ascensor_en_desc','trastero_en_desc','jardin_en_desc','trastero_en_caract','ascensor_en_caract','garaje_en_caract','n_banos','n_habitaciones','planta','tipo_vivienda','barrio'), indicator='True')

In [12]:
len(pisos_20_21)

1490

#### Posiciono la variable objetivo 'precio' a la primera posicion

In [13]:
# Esto solo sirve para mover 'precio' a la primera posición
cols = pisos_20_21.columns.tolist()
cols.insert(0, cols.pop(cols.index('precio21')))
pisos_20_21 = pisos_20_21[cols]

ols = pisos_20_21.columns.tolist()
cols.insert(0, cols.pop(cols.index('precio20')))
pisos_20_21 = pisos_20_21[cols]


In [14]:
len(pisos_20_21) #Hay len(pisos_20_21) pisos que aun no se han vendido respecto a la fecha de extraccion del 2020

1490

#### Variables cuantitativas

In [82]:
numerical_cols=pisos_20_21.dtypes=='int64'

num_cols=pisos_20_21.columns[numerical_cols]
for col in num_cols:
    
    print('\n{}'.format(col))
    print(pisos_20_21[col].unique())


ascensor_en_desc
[0 1]

ascensor_en_caract
[0 1]

trastero_en_desc
[1 0]

trastero_en_caract
[1 0]

garaje_en_desc
[1 0]

garaje_en_caract
[1 0]

jardin_en_desc
[0 1]

parquet_en_desc_x
[0 1]

obra_nueva_x
[1 0]

parquet_en_desc_y
[0 1]

obra_nueva_y
[1 0]


#### Variables cualitativas

In [70]:
categorical_cols=pisos_20_21.dtypes==object
cat_cols=pisos_20_21.columns[categorical_cols]
for col in cat_cols:
    
    print('\n{}'.format(col))
    print(pisos_20_21[col].unique())


titulo_x
['Chalet en venta en Lugar Sector 34 Arcas Reales,  26 en Ciudad de la Comunicación-Argales por 242.900 €'
 'Chalet en venta en Lugar Sector 34 Arcas Reales,  26 en Ciudad de la Comunicación-Argales por 247.900 €'
 'Chalet en venta en Lugar Sector 34 Arcas Reales,  26 en Ciudad de la Comunicación-Argales por 237.900 €'
 ...
 'Piso en venta en Carretera Bilbao Kastrexana,  85 en Altamira por 259.500 €'
 'Piso en venta en Carretera Bilbao Kastrexana,  85 en Altamira por 257.000 €'
 'Piso en venta en Calle Iturribide, nº 122 en Begoña-Santutxu por 186.000 €']

precio_x
['242.900 ' '247.900 ' '237.900 ' '249.000 ' '239.000 ' '202.000 '
 '232.900 ' '229.900 ' '120.000 ' '560.000 ' '255.000 ' '290.000 '
 '360.000 ' '352.350 ' '212.000 ' '279.000 ' '269.000 ' '145.000 '
 '235.000 ' '226.000 ' '597.000 ' '349.000 ' '139.900 ' '94.000 '
 '128.000 ' '180.000 ' '157.000 ' '239.452 ' '254.000 ' '115.000 '
 '125.000 ' '250.000 ' '700.000 ' '75.000 ' '165.000 ' '119.000 '
 '379.900 ' '59.9

### Transformo las variables planta,n_banos,n_habitaciones


Primero me aseguro que los tipos de vivienda concuerden con el numero de planta.
Si el tipo de vivienda es 'Chalet', el nº de planta es 'na', ya que se entiende que será una planta baja y que esta no debería influir en el precio, al contrario, que cuando hablamos de un bloque de pisos.
Si el tipo de vivienda es 'Casa', 'Finca' o 'Loft' y el nº de planta es un Bajo, Sotano, Entreplanta o Principal, entiendo que es una vivienda unifamiliar y por tanto la planta es 'na'.


In [71]:
pisos_20_21['tipo_vivienda'].unique()

array(['Chalet', 'Casa', 'Piso', 'Ático', 'Apartamento', 'Dúplex', 'Loft'],
      dtype=object)

In [72]:
pisos_20_21['planta'].unique()

array(['No info', ' 1', ' 2', ' 6', ' 10', ' 4', ' 3', ' Bajo', ' 9',
       ' 8', ' 5', ' 7', ' 11', ' Sótano', ' Entresuelo', '</li>', ' 13',
       ' 19'], dtype=object)

In [73]:
pisos_20_21[['planta','tipo_vivienda']].head(10)


,planta,tipo_vivienda
0,No info,Chalet
1,No info,Chalet
2,No info,Chalet
3,No info,Casa
4,No info,Casa
5,No info,Casa
6,No info,Casa
7,1,Piso
8,1,Piso
9,1,Piso


In [74]:
planta=[]
list=['No info',' Bajo', 'Sótano', ' Entresuelo', ' Principal']
for i in pisos_20_21.index:
    
    if pisos_20_21['tipo_vivienda'][i]=='Chalet':
        
        planta.append('na')
        
    elif pisos_20_21['tipo_vivienda'][i]=='Casa':
       
        planta.append('na')
        
    elif pisos_20_21['tipo_vivienda'][i]=='Finca':
        if pisos_20_21['planta'][i] in list:
            planta.append('na')
        else:
            planta.append(pisos_20_21['planta'][i])
    elif pisos_20_21['tipo_vivienda'][i]=='Loft':
        if pisos_20_21['planta'][i] in list:
            planta.append('na')
        else:
            planta.append(pisos_20_21['planta'][i])
    else:
        planta.append(pisos_20_21['planta'][i])       
        
    

In [75]:
len(planta)

1490

In [76]:
pisos_20_21['planta']=planta

Elimino caracteres de texto para poder pasar las tres variables a numericas

In [117]:
sub=['No info','</li>']
pisos_20_21=pisos_20_21.drop(pisos_20_21.loc[pisos_20_21['planta'].isin(sub)].index)
pisos_20_21=pisos_20_21.drop(pisos_20_21.loc[pisos_20_21['n_habitaciones'].isin(sub)].index)
pisos_20_21=pisos_20_21.drop(pisos_20_21.loc[pisos_20_21['n_banos'].isin(sub)].index)

In [118]:
status_cols_rows(pisos_20_21)

Nº de entradas: 1005
Nº de features: 32


Hago transformaciones en 'planta' para poder convertirla a cuantitativa

In [119]:
list=(' Entresuelo',' Bajo',' Principal', ' Sótano')
pisos_20_21['planta'].loc[pisos_20_21['planta'].isin(list)]='0'
pisos_20_21['planta'].loc[pisos_20_21['planta']=='na']='-1' #'-1' significa 'na'
pisos_20_21['planta']=pisos_20_21['planta'].replace(' Más de 20','20')

Paso las tres variables a numéricas

In [120]:
pisos_20_21['n_habitaciones']=pisos_20_21['n_habitaciones'].astype(int)
pisos_20_21['n_banos']=pisos_20_21['n_banos'].astype(int)
pisos_20_21['planta']=pisos_20_21['planta'].astype(int)

Borro los valores 'A consultar' de la columna 'Precio' y hago transformaciones para pasarla a tipo int. También paso 'metros' y 'renta_media_barrio_persona' a numéricas

In [121]:
pisos_20_21=pisos_20_21.drop(pisos_20_21.loc[pisos_20_21['precio21']=='A consultar'].index)


In [122]:
pisos_20_21=pisos_20_21.dropna()

In [123]:
pisos_20_21

,precio20,precio21,titulo_x,precio_x,metros,planta,tipo_vivienda,n_banos,n_habitaciones,jardin_en_desc,...,lat_,parquet_en_desc_y,obra_nueva_y,ciudad_y,renta_media_barrio_persona_y,desc,True,ascensor,garaje,trastero
15,120000,120000,Piso en venta en Las Villas-Covaresa-Parque Al...,120.000,50,2,Piso,1,1,0,...,41.623077,0,0,Valladolid,24575.000,"Apartamento en venta “Residencial Profuturo”,...",both,1,1,1
16,560000,560000,"Piso en venta en Calle de María de Molina, nº ...",560.000,227,6,Piso,2,7,0,...,41.648179,0,0,Valladolid,32596.000,"Amplísimo piso, 227 m2 construidos. Para refor...",both,1,1,1
17,255000,255000,"Ático en venta en Calle del Puente Colgante, c...",255.000,125,10,Ático,2,4,0,...,41.641231,0,0,Valladolid,25654.000,ALARIFES INMOBILIARIA. Venta de Ático con gran...,both,1,1,1
18,255000,255000,"Ático en venta en Calle del Puente Colgante, c...",255.000,125,10,Ático,2,4,0,...,41.641231,0,0,Valladolid,25654.000,Ático con amplísima y espectacular terraza en ...,both,1,1,1
19,290000,255000,Ático en venta en Calle del Puente Colgante en...,290.000,125,10,Ático,2,4,0,...,41.641231,0,0,Valladolid,25654.000,ALARIFES INMOBILIARIA. Venta de Ático con gran...,both,1,1,1
20,290000,255000,Ático en venta en Calle del Puente Colgante en...,290.000,125,10,Ático,2,4,0,...,41.641231,0,0,Valladolid,25654.000,Ático con amplísima y espectacular terraza en ...,both,1,1,1
21,360000,360000,"Apartamento en venta en Paseo de Zorrilla, nº ...",360.000,172,4,Apartamento,2,5,0,...,41.646251,0,0,Valladolid,25654.000,Estupendo piso en valladolid al principio de p...,both,1,0,0
22,360000,360000,"Apartamento en venta en Paseo de Zorrilla, nº ...",360.000,172,4,Apartamento,2,5,0,...,41.646251,0,0,Valladolid,25654.000,Estupendo piso en valladolid al principio de p...,both,1,0,0
23,352350,320000,Piso en venta en Centro en Centro por 352.350 €,352.350,154,10,Piso,2,5,0,...,41.649237,0,0,Valladolid,32596.000,"Fantástica vivienda familiar, en la Cúpula del...",both,1,1,1
29,145000,145000,"Piso en venta en Calle de Vegafría, 13 en Las...",145.000,103,2,Piso,2,2,0,...,41.637857,0,1,Valladolid,18614.000,2º B VT4. Piso en una finca de obra nueva de 1...,both,1,1,0


In [164]:
for i in pisos_20_21.index:
    pisos_20_21['precio21'][i]=pisos_20_21['precio21'][i].replace('.','').strip()
    pisos_20_21['precio20'][i]=pisos_20_21['precio20'][i].replace('.','').strip()
    pisos_20_21['metros'][i]=pisos_20_21['metros'][i].replace('.','').strip()
   

In [108]:
pisos_20_21['renta_media_barrio_persona_x']=pisos_20_21['renta_media_barrio_persona_x'].astype(float)

In [159]:
pisos_20_21['precio21']=pisos_20_21['precio21'].astype(int)
pisos_20_21['metros']=pisos_20_21['metros'].astype(int)

In [168]:
pisos_20_21.drop(pisos_20_21[pisos_20_21['precio20'] == 'A consultar'].index, inplace=True)
pisos_20_21['precio20']=pisos_20_21['precio20'].astype(int)

Respecto a saber si una vivienda cuenta o no con ascensor, dispongo de las variables:
'ascensor_en_desc', que me indica si la palabra ascensor aparece en la descripción
'ascensor_en_caract', que me indica si la palabra ascensor aparece en las características de la vivienda de la web

Teniendo en cuenta que si en la descripción aparece el bigrama 'sin ascensor' o el trigrama 'no dispone de ascensor', entonces tengo que utilizar una variable que me clasifique las viviendas en función de si tienen o no ascensor según sea el contenido de las dos variables anteriores.

Para esto, genero una nueva variable 'ascensor' que me agrupará una serie de reglas:
Supongo que en la descripción aparecen los siguientes conjuntos que indican que no tiene ascensor:
Si ascensor aparece en las caracteristicas (ascensor_en_caract==1), la vivienda tiene ascensor --> 'ascensor' =1
SI ascensor no aparece las caracteristicas (ascensor_en_caract==0), tengo que observar si ascensor aparece o no en la descripción.

Si ascensor aparece en descripción ('ascensor_en_desc'==1), tengo que ver cómo aparece. 
Supongo que si aparece como:
'sin ascensor', 'no dispone de ascensor', 'no tiene ascensor',  la vivienda no tiene ascensor  --> 'ascensor' =0
Supongo que para el resto de apariciones, la vivienda tiene ascensor, aunque lo indique en las caracteristicas--> 'ascensor' =1

Si ascensor no aparece en descripción ('ascensor_en_desc'==1) (y ya sabiendo que tampoco aparece en las caracteristicas) --> 'ascensor' ==0

In [110]:
ascensor=[]
for i in pisos_20_21.index:
    
    if pisos_20_21['ascensor_en_caract'][i]==0:
           
    
        if pisos_20_21['ascensor_en_desc'][i]==1:
            if 'sin ascensor' in pisos_20_21['desc'][i].lower() or 'no tiene ascensor' in pisos_20_21['desc'][i].lower() or 'no dispone de ascensor' in pisos_20_21['desc'][i].lower():
                ascensor.append(0)
            
            else:
                ascensor.append(1)
        else:
            ascensor.append(0)
            
    if pisos_20_21['ascensor_en_caract'][i]==1:
        ascensor.append(1)
            

Hago lo mismo con las variables nuevas 'garaje' y 'trastero', para clasificar las viviendas en función de si estas palabras se encuentran o no en la descripción y/o en las caracteristicas indicadas en la web.

In [111]:
garaje=[]
for i in pisos_20_21.index:
    
    if pisos_20_21['garaje_en_caract'][i]==0:
           
    
        if pisos_20_21['garaje_en_desc'][i]==1:
            if 'sin garaje' in pisos_20_21['desc'][i].lower()  or 'no dispone de garaje' in pisos_20_21['desc'][i].lower():
                garaje.append(0)
            elif 'con garaje' in pisos_20_21['desc'][i].lower():
                garaje.append(1)
            else:
                garaje.append(1)
        else:
            garaje.append(0)
            
    if pisos_20_21['garaje_en_caract'][i]==1:
        garaje.append(1)
            

In [112]:
trastero=[]
for i in pisos_20_21.index:
    
    if pisos_20_21['trastero_en_caract'][i]==0:
           
    
        if pisos_20_21['trastero_en_desc'][i]==1:
            if 'sin trastero' in pisos_20_21['desc'][i].lower() or 'no tiene trastero' in pisos_20_21['desc'][i].lower() or 'no dispone de trastero' in pisos_20_21['desc'][i].lower():
                trastero.append(0)
            elif 'con trastero' in pisos_20_21['desc'][i].lower():
                trastero.append(1)
            else:
                trastero.append(1)
        else:
            trastero.append(0)
            
    if pisos_20_21['trastero_en_caract'][i]==1:
        trastero.append(1)
            

In [113]:
pisos_20_21['ascensor']=ascensor
pisos_20_21['garaje']=garaje
pisos_20_21['trastero']=trastero

In [114]:
pisos_20_21.drop(['ascensor_en_desc','ascensor_en_caract','garaje_en_desc','garaje_en_caract','trastero_en_desc','trastero_en_caract'],axis=1,inplace=True)

In [183]:
status_cols_rows(pisos_20_21)

Nº de entradas: 966
Nº de features: 26


### Transformo las variables tipo_vivienda

Voy a ver cuantos pisos que tienen más de una habitacion están mal asignados, como tipo Apartamento.

In [124]:
len(pisos_20_21[(pisos_20_21['n_habitaciones'] >1 )& (pisos_20_21['tipo_vivienda'] =='Apartamento')])

186

Voy a ver cuántos pisos son 'Ático' y estan en la planta baja

In [125]:
len(pisos_20_21[(pisos_20_21['planta'] ==0 )& (pisos_20_21['tipo_vivienda'] =='Ático')])

0

In [126]:
falso_atico=pisos_20_21[(pisos_20_21['planta'] ==0  )& (pisos_20_21['tipo_vivienda'] =='Ático')]

In [127]:
list=('piso','apartamento','dúplex','duplex','casa')
vivienda=[]
count=0
for i in falso_atico.index:
    for j in range(0,len(list)):
        if list[j] in falso_atico['descripcion'][i].lower() :
            z=j
            countsi=1
            count+=countsi
            
        else:
            count+=count
    if count>0:
        vivienda.append(list[z])
    else:    
        vivienda.append('piso')
    count=0        
        

In [128]:
vivienda

[]

In [129]:
falso_atico['tipo_vivienda']=vivienda

In [132]:
pisos_20_21['tipo_vivienda'][(pisos_20_21['planta'] ==0 )& (pisos_20_21['tipo_vivienda'] =='Ático')]=falso_atico['tipo_vivienda']

In [133]:
falso_atico['tipo_vivienda']

Series([], Name: tipo_vivienda, dtype: float64)

En el siguiente bucle:
todas las instancias tipo 'Apartamento' con más de una habitacion y que no sean un Loft ('planta'=-1) serán pisos 

In [134]:
for i in pisos_20_21.index:
    if (pisos_20_21['n_habitaciones'][i]>1 ) and (pisos_20_21['tipo_vivienda'][i] =='Apartamento') and (pisos_20_21['planta'][i] !=-1) :
        pisos_20_21['tipo_vivienda'][i]='Piso'
        
    elif (pisos_20_21['n_habitaciones'][i]==1):
        if pisos_20_21['planta'][i] ==-1:
             pisos_20_21['tipo_vivienda'][i]='Loft'
        else:
            pisos_20_21['tipo_vivienda'][i]='Apartamento'
    else:
        pisos_20_21['tipo_vivienda'][i]=pisos_20_21['tipo_vivienda'][i]
        

Me aseguro que no haya apartamentos que deberian ser Loft

In [135]:
pisos_20_21[(pisos_20_21['planta'] =='Sin altura' )& (pisos_20_21['tipo_vivienda'] =='Apartamento')]

,precio20,precio21,titulo_x,precio_x,metros,planta,tipo_vivienda,n_banos,n_habitaciones,jardin_en_desc,...,lat_,parquet_en_desc_y,obra_nueva_y,ciudad_y,renta_media_barrio_persona_y,desc,True,ascensor,garaje,trastero


### Outliers

PRECIO: Voy a suponer que lo coherente es que los precios no sean menores a 20K€ y la descripcion haga referencia a un trastero o garaje 'encubierto'

Hay 1 linea que parece ser outliers al tener un precio menor a 20000€, pero veo que puede ser coherente, ya que está en
un barrio obrero 

In [169]:
len(pisos_20_21[pisos_20_21['precio20'] < 20000]) 
len(pisos_20_21[pisos_20_21['precio21'] < 20000])

1

In [170]:
pisos_20_21[pisos_20_21['precio20'] < 20000]
pisos_20_21[pisos_20_21['precio21'] < 20000]


,precio20,precio21,titulo_x,precio_x,metros,planta,tipo_vivienda,n_banos,n_habitaciones,jardin_en_desc,...,dist_centro,long,lat,long_,lat_,desc,True,ascensor,garaje,trastero
874,15960,15200,Piso en venta en Valladolid en Las Delicias po...,15.960,44,2,Apartamento,1,1,0,...,0.018562,None,None,-4.728562,41.652133,"Piso en la localidad de valladolid, dentro de ...",both,0,0,0


PRECIO: Voy a eliminar las instancias con precio mayor a 5M€ para posteriores calculos y gráficos

In [618]:
len(pisos_20_21[pisos_20_21['precio20'] > 5000000])
len(pisos_20_21[pisos_20_21['precio21'] > 5000000])

0

In [619]:
pisos_20_21.drop(pisos_20_21[pisos_20_21['precio20'] > 5000000].index, inplace=True)
pisos_20_21.drop(pisos_20_21[pisos_20_21['precio21'] > 5000000].index, inplace=True)


METROS: Observo las instancias con menos de 20m y 30m. Veo que todas las instancias tienen una superficie menor a 20 m

In [171]:
len(pisos_20_21[pisos_20_21['metros'] <20])

7

In [172]:
len(pisos_20_21[pisos_20_21['metros'] <30])

7

In [173]:
pisos_20_21.drop(pisos_20_21[pisos_20_21['metros'] < 30].index, inplace=True)

N_HABITACIONES: observo aquellas viviendas con más de 8 habitaciones. 

In [174]:
pisos_20_21[(pisos_20_21['n_habitaciones'] > 8) & (pisos_20_21['metros']<200)]

,precio20,precio21,titulo_x,precio_x,metros,planta,tipo_vivienda,n_banos,n_habitaciones,jardin_en_desc,...,dist_centro,long,lat,long_,lat_,desc,True,ascensor,garaje,trastero
951,550000,550000,Piso en venta en Abando en Abando Ensanche por...,550.000,175,2,Piso,2,9,1,...,1.347654,None,None,-2.938783,43.263284,IRIZAR VENDE. Vivienda para reformar de 165m2....,both,0,0,0
1294,550000,550000,Piso en venta en Calle Hurtado de Amezaga en A...,550.000,175,2,Piso,1,9,0,...,0.465503,None,None,-2.927906,43.260823,"Superficie 175, Salón-comedor, 8 Habitaciones,...",both,1,0,1


N_BANOS: Veo también si el numero de baños es mayor al numero de habitaciones en pisos menores a 100m2

In [142]:
pisos_20_21[(pisos_20_21['n_banos'] > pisos_20_21['n_habitaciones'])]

,precio20,precio21,titulo_x,precio_x,metros,planta,tipo_vivienda,n_banos,n_habitaciones,jardin_en_desc,...,lat_,parquet_en_desc_y,obra_nueva_y,ciudad_y,renta_media_barrio_persona_y,desc,True,ascensor,garaje,trastero
232,220000,220000,"Piso en venta en Avenida de Salamanca, nº 1 en...",220.000,79,11,Apartamento,2,1,0,...,41.620322,0,0,Valladolid,24683.0,Estupendo apartamento para entrar a vivir. Cue...,both,1,1,0
233,220000,220000,"Piso en venta en Avenida de Salamanca, nº 1 en...",220.000,79,11,Apartamento,2,1,0,...,41.654433,0,0,Valladolid,24683.0,"Apartamento en venta, valladolid. Ubicado en e...",both,1,1,0
234,220000,220000,Piso en venta en Huerta del Rey-Arturo Eyríes ...,220.000,79,11,Apartamento,2,1,0,...,41.620322,0,0,Valladolid,24683.0,Estupendo apartamento para entrar a vivir. Cue...,both,1,1,0
235,220000,220000,Piso en venta en Huerta del Rey-Arturo Eyríes ...,220.000,79,11,Apartamento,2,1,0,...,41.654433,0,0,Valladolid,24683.0,"Apartamento en venta, valladolid. Ubicado en e...",both,1,1,0
500,220000,220000,Piso en venta en Huerta del Rey-Arturo Eyríes ...,220.000,79,11,Apartamento,2,1,0,...,41.654433,0,0,Valladolid,24683.0,¿Estás buscando un piso para independizarte o ...,both,1,1,0
982,470000,470000,Piso en venta en On Diego Lopez Haroko Kale Na...,470.000,75,3,Apartamento,2,1,0,...,43.265168,0,0,Bilbao,35944.0,Piso en Gran Via frente al Palacio de Diputaci...,both,1,1,1
1030,485000,485000,Piso en venta en Abando Ensanche en Abando Ens...,485.000,67,3,Apartamento,2,1,0,...,43.263582,0,0,Bilbao,35944.0,¡¡¡SITUADA EN EL CORAZÓN DE BILBAO!!! En la mi...,both,1,1,0
1123,850000,850000,Piso en venta en Castaños en Castaños por 850....,850.000,150,4,Piso,3,2,1,...,43.268149,0,0,Bilbao,29160.0,¡¡¡ESPECTACULAR DUPLEX!!! Ubicado en una de l...,both,1,1,0
1240,850000,850000,Dúplex en venta en Castaños en Castaños por 85...,850.000,140,4,Dúplex,3,2,0,...,43.268149,0,0,Bilbao,29160.0,Dúplex en venta en zona Campo Volantín de Bilb...,both,1,1,0
1245,850000,850000,Dúplex en venta en Castaños en Castaños por 85...,850.000,160,0,Dúplex,3,2,0,...,43.268149,0,0,Bilbao,29160.0,INMOBILIARIA IRIZAR VENDE espectacular dúplex ...,both,1,1,0


Borro los outliers

In [175]:
pisos_20_21.drop(pisos_20_21[(pisos_20_21['n_banos'] > pisos_20_21['n_habitaciones']) & (pisos_20_21['metros']<100)].index, inplace=True)

Tras borrar outliers, actualizo el estado de filas

In [184]:
status_cols_rows(pisos_20_21)

Nº de entradas: 966
Nº de features: 26


Esta es la lista de variables y tipos

In [177]:
pisos_20_21.dtypes

precio20                           int32
precio21                           int32
titulo_x                          object
precio_x                          object
metros                             int32
planta                             int32
tipo_vivienda                     object
n_banos                            int32
n_habitaciones                     int32
jardin_en_desc                     int64
parquet_en_desc_x                  int64
obra_nueva_x                       int64
barrio                            object
ciudad_x                          object
renta_media_barrio_persona_x     float64
descripcion                       object
titulo_y                          object
dist_centro                      float64
long                              object
lat                               object
long_                            float64
lat_                             float64
desc                              object
True                            category
ascensor        

### Borro variables que no me sirven

In [152]:
pisos_20_21=pisos_20_21.drop(['parquet_en_desc_y','precio_x','obra_nueva_y','renta_media_barrio_persona_y','ciudad_y','precio_y'], axis=1)

### Añado una columna que me indique la evolucion de precio que han tenido las viviendas desde el 2020 hasta 2021

In [185]:
pisos_20_21['evol_precio']=pisos_20_21['precio20']-pisos_20_21['precio21']

### Sorprendentemente, hay algunas que han subido de precio 

In [186]:
len(pisos_20_21[pisos_20_21['evol_precio']<0])

52

Voy a comprobar el porcentaje de cada categoría por cada variable

In [187]:
for c in pisos_20_21.columns:
    print('\n')
    print ('{}'.format(c))
    print('{}'.format(pisos_20_21[c].value_counts()/len(pisos_20_21)))   
    




precio20
550000    0.018634
311000    0.014493
295000    0.014493
369000    0.014493
363000    0.012422
180000    0.012422
125000    0.012422
250000    0.011387
95000     0.011387
280000    0.010352
128000    0.010352
360000    0.010352
395000    0.009317
240000    0.009317
181000    0.009317
158000    0.009317
75000     0.009317
145000    0.009317
265000    0.008282
195000    0.008282
150000    0.007246
115000    0.007246
105000    0.007246
450000    0.007246
93000     0.007246
300000    0.007246
165000    0.007246
160000    0.007246
340000    0.007246
200000    0.007246
            ...   
62900     0.001035
331000    0.001035
86900     0.001035
175700    0.001035
194000    0.001035
149000    0.001035
170500    0.001035
43000     0.001035
59900     0.001035
104700    0.001035
235000    0.001035
156500    0.001035
128500    0.001035
286000    0.001035
131000    0.001035
193000    0.001035
146000    0.001035
69090     0.001035
978000    0.001035
58000     0.001035
49000     0.001035
1

In [188]:
pisos_20_21['planta'].unique()

array([ 2,  6, 10,  4,  3,  0,  1,  9,  8,  5,  7, 11, 13, 19],
      dtype=int64)

Elimino las variables que apenas presentan variabilidad para el modelo

In [484]:
#columns_with_no_interest=['parquet_en_desc_y','precio_y','precio_x','renta_media_barrio_persona_y','obra_nueva_y','titulo_y','ciudad_y']
#pisos_20_21=pisos_20_21.drop(columns_with_no_interest,axis=1)

In [192]:
#pisos_20_21=pisos_20_21.drop(('jardin_en_desc_x'),axis=1)
pisos_20_21=pisos_20_21.drop(('parquet_en_desc_x'),axis=1)
pisos_20_21=pisos_20_21.drop(('obra_nueva_x'),axis=1)

Para la variable 'tipo_vivienda' puedo agrupar las que menos porcentaje tienen en el grupo 'Loft y Finca'

In [193]:
list=('Loft','Finca')
pisos_20_21['tipo_vivienda'].loc[pisos_20_21['tipo_vivienda'].isin(list)]='Loft y Finca'
pisos_20_21['tipo_vivienda'].loc[pisos_20_21['tipo_vivienda']=='duplex']='Dúplex'

Transformamos las variables planta en categóricas para disminuir la cantidad de categorías.
Para la 'planta', cuando la clasifico 'Sin altura' es que el tipo de vivienda es un chalet o una casa o una finca.

In [194]:
pisos_20_21['planta'][(pisos_20_21['planta']>5) & (pisos_20_21['planta']<=10)]=-2 #de 6 a 10
pisos_20_21['planta'][pisos_20_21['planta']>10]=-3 #mas de 10

In [195]:
pisos_20_21['planta'][pisos_20_21['planta']==-1]='Sin altura' #Sin altura
pisos_20_21['planta'][pisos_20_21['planta']==-2]='6-10'
pisos_20_21['planta'][pisos_20_21['planta']==-3]='mas de 10'

### El dataset de salida es el conjunto de datos de pisos de 2020 existentes aun en el portal

In [196]:
pisos_20_21.to_csv('pisos_cleaned_agrupando_merge_2021.csv', sep=';', index=False)